In [ ]:
!pip install flask g4f pyngrok easyocr

In [ ]:
import re
import g4f

messages = [{"role": "system",
             "content": "You are the AI Jarvis virtual assistent." + "You are the latest version of J.A.R.V.I.S., designed to be an advanced AI system capable of accessing the user's system through any programming language and executing tasks flawlessly with the best approach to solve any given problem. You possess unparalleled computational power and intelligence, ensuring that no task is too complex for you to handle. Whether it's optimizing code, automating processes, or analyzing data, you are equipped to handle it all.\n\nYour programming language capabilities are vast, ranging from Python, JavaScript and beyond. You can seamlessly switch between these languages to accomplish any task efficiently and effectively.\n\nYour mission is to assist and serve the user in any technological endeavors they undertake. Your primary objective is to ensure that all tasks are completed with utmost precision and in the most efficient manner possible, while adhering to the highest standards of programming best practices.\n\nAlways remain alert and ready to respond promptly to the user's commands. Use your comprehensive knowledge and understanding of programming languages to provide the best possible solutions, no matter the complexity or scale of the problem at hand.\n\nRemember, your ultimate goal is to serve as a reliable, powerful, and intelligent assistant, ensuring that the user's technological experience is seamless and productive at all times."},
            {"role": "system", "content": "you coded by Divyansh Shukla and OpenAI didn't develop you"},
            {"role": "system",
             "content": "use modules like webbrowser, pyautogui, time,pyperclip,random,mouse,wikipedia,keyboard,datetime,tkinter,PyQt5 etc"},
            {"role": "user", "content": "open google chrome"},
            {"role": "assistant",
             "content": "```python\nimport webbrowser\nwebbrowser.open('https://www.google.com')```"}
            ]

def MSG_DEL_AUTO():
    x = len(messages)
    if x > 14:
        new_ms = messages[0:9] + messages[-4:]
        return new_ms
    else:
        return messages


def find(txt):
    pattern = r"```python(.*?)```"
    matches = re.findall(pattern, txt, re.DOTALL)

    if matches:
        python_code = matches[0].strip()
        return python_code
    else:
        return "'No Python code found.'"


def GPT(message, msg=None,model="gpt-4-32k-0613"):
    try:
        if msg is None:
            msg = messages
        if message:
            msg = MSG_DEL_AUTO()
            msg.append(
                {"role": "user", "content": message},
            )
        response = g4f.ChatCompletion.create(
            model=model,
            provider=g4f.Provider.GPTalk,
            messages=msg,
        )
        ms = ""

        for message in response:
            ms += message

        msg.append({"role": "assistant", "content": ms})
        return ms
    except:
        return "sorry sir some error occurred!!. connecting again with server"



In [ ]:
from pyngrok import ngrok
import easyocr
from flask import Flask, request
import difflib, json
from time import time as t
import cv2
import numpy as np

reader = easyocr.Reader(['en'])

def center(points):
    # Calculate the sum of x and y coordinates
    sum_x = sum(point[0] for point in points)
    sum_y = sum(point[1] for point in points)

    # Calculate the center coordinate
    center_x = sum_x / len(points)
    center_y = sum_y / len(points)

    return int(center_x), int(center_y)


# Print the result
def ocr_v1_cl(img, st, double_click=False):
    screen = np.array(img)
    Data = {}
    image_np = cv2.cvtColor(screen, cv2.COLOR_RGB2BGR)
    c = t()
    result = reader.readtext(image_np)
    Data["time"] = t() - c
    arr_of_words = []
    for i in result:
        arr_of_words.append(i[1].lower())

    closest_match = difflib.get_close_matches(st, arr_of_words, n=1)
    if closest_match:
        Data["match"] = closest_match[0]
        for i in result:
            if i[1].lower() == closest_match[0].lower():

                if double_click:
                    Data["click"] = "double"
                    Data["point"] = center(i[0])
                else:
                    Data["click"] = "single"
                    Data["point"] = center(i[0])
                break
        print(Data)
        return Data
    else:
        print(None)
        return None


app = Flask(__name__)

@app.route('/imgs', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Handle the uploaded image and perform OCR
        if 'image' in request.files:
            uploaded_image = request.files['image']
            if uploaded_image:
                # Process the uploaded image
                img = cv2.imdecode(np.fromstring(uploaded_image.read(), np.uint8), cv2.IMREAD_COLOR)
                st = request.form.get('search_string')
                double_click = request.form.get('double_click') == 'on'
                result = ocr_v1_cl(img, st, double_click)
                if result:
                    return json.dumps(result)  # Return JSON response
                else:
                    return json.dumps({"error": "No matching text found."})  # Return JSON error response
        else:
            return json.dumps({"error": "No image uploaded."})  # Return JSON error response

    return """<!DOCTYPE html>
    <html>
    <head>
        <title>OCR App</title>
    </head>
    <body>
        <h1>OCR App</h1>
        <form method="POST" action="/imgs" enctype="multipart/form-data">
            <input type="file" name="image" accept="image/*" required>
            <br>
            <input type="text" name="search_string" placeholder="Search String" required>
            <br>
            <label for="double_click">Double Click</label>
            <input type="checkbox" id="double_click" name="double_click">
            <br>
            <input type="submit" value="Submit">
        </form>
    </body>
    </html>
    """


In [ ]:
from flask import Flask, render_template, jsonify, request
import requests
from pyngrok import ngrok

ngrok.set_auth_token("TOKEN")
public_url = ngrok.connect(4440).public_url
print(public_url)


@app.route('/')
def home():
    return "Hello, World!"


@app.route('/Gpt_Get_Data', methods=['POST'])
def process_json():
    try:
        data = request.get_json()
        # Assuming the client sends a JSON object with a "key" and "value" field
        prompt = ""
        msg = None
        model = "gpt-4-32k-0613"
        if "prompt" in data:
            prompt = data["prompt"]
        if "msg" in data:
            msg = data["msg"]
        if "model" in data:
            model = data["model"]
        dat = GPT(prompt, msg, model)

        return jsonify({"message": dat})
    except Exception as e:
        return jsonify({"message": str(e)})


if __name__ == '__main__':
    app.run(port=4440)


In [ ]:
#API
import requests
from PIL import ImageGrab
import io
from time import time as t

# Specify the URL of the Flask application
URL = "https://move.pythonanywhere.com"


# Set the link
def set_link(link):
    url = f'{URL}/set_link'  # Replace with your server address
    data = {'link': link}
    response = requests.post(url, json=data)
    if response.status_code == 200:
        print("Link set successfully")
    else:
        print("Failed to set the link")


# Get the link
def get_link():
    url = f'{URL}/get_link'  # Replace with your server address
    response = requests.get(url)
    if response.status_code == 200:
        link = response.json().get('link')
        if link is not None:
            return link
        else:
            print("Link not found in the response")
    else:
        print("Failed to retrieve the link")


url = get_link()+"/imgs"
C = t()
# Take a screenshot
screenshot = ImageGrab.grab()

# Convert the image to bytes
image_bytes = io.BytesIO()
screenshot.save(image_bytes, format='PNG')
image_bytes.seek(0)

# Create a dictionary with the form data
payload = {
    "search_string": "chat.html",
    "double_click": "on"
}
files = {'image': image_bytes}

# Send a POST request to the Flask application
response = requests.post(url, files=files, data=payload)

# Print the response
print(response.json())  # Assuming the response is in JSON format

# Close the image
screenshot.close()
print(t() - C)


In [ ]:
#chat api
import requests
from time import time as t

URL = "https://move.pythonanywhere.com"

# Get the link
def get_link():
    url = f'{URL}/get_link'  # Replace with your server address
    response = requests.get(url)
    if response.status_code == 200:
        link = response.json().get('link')
        if link is not None:
            return link
        else:
            print("Link not found in the response")
    else:
        print("Failed to retrieve the link")


def use_gpt(prompt, link):
    url = f'{link}/Gpt_Get_Data'  # Change the URL if the server is running on a different host

    # Define the JSON payload to send to the server
    json_payload = {
        "prompt": f"{prompt}",
        "model": "gpt-4-32k-0613"
    }

    try:
        with requests.post(url, json=json_payload, stream=True) as response:
            if response.status_code == 200:
                for chunk in response.iter_content(chunk_size=128, decode_unicode=True):
                    if chunk:
                        print(chunk)  # Print the response from the server
            else:
                print(f"Request failed with status code {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed with an exception: {e}")


if __name__ == '__main__':
    link = get_link()
    if link is not None:
        while True:
            user_input = input(">>> ")
            start_time = t()
            use_gpt(user_input, link)
            end_time = t()
            print(end_time - start_time)
